In [58]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import math

In [59]:
def get_conn(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [60]:
def get_df_from_sql(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [61]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'
query = """with ops_main as (
    select
        awb
    ,   created_date
    from public.ops_main
    where 1=1
    and shipping_partner = 'Hyperlocal'
    and date_trunc('day', created_date) >= date_trunc('day', now() + interval'5.5 hours' - interval'1 day')
)
,
shipment_order_details as (
    select
        awb
    ,   warehouse_id
    from public.shipment_order_details
)
,
base as (
    select
        s.warehouse_id
    ,   o.created_date
    ,   o.awb
    from ops_main o
    left join shipment_order_details s on o.awb = s.awb
    order by 1, 2
)

select * from base"""

# Establish a connection
conn = get_conn(SSH_required, key_path)

# Retrieve data into a DataFrame
df = get_df_from_sql(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame 'df'
print(df)

Tunnel Started
Connection Made
       warehouse_id               created_date           awb
0                83 2024-07-07 08:07:29.551418  GS1362278349
1                83 2024-07-07 08:10:26.063166  GS1054164984
2                83 2024-07-07 08:19:34.502561  GS1331160028
3                83 2024-07-07 08:25:34.501822  GS1961716865
4                83 2024-07-07 08:34:30.164179  GS1301848766
...             ...                        ...           ...
40580          2147 2024-07-08 15:45:33.673427  GS1176585047
40581          2151 2024-07-07 10:39:50.042359  GS1263341845
40582          2151 2024-07-07 17:10:11.244005  GS1593016996
40583          2151 2024-07-08 12:39:30.286308  GS1044316150
40584          2151 2024-07-08 16:54:51.623534  GS1255804329

[40585 rows x 3 columns]


In [62]:
df['created_date'] = pd.to_datetime(df['created_date'])

In [63]:
file_path = 'pickup_slots.xlsx'
results_df = pd.read_excel(file_path)

In [64]:
expected_loads = []

for warehouse, slot in results_df.groupby('warehouse_id'):
    # slot = slot.reset_index(drop=True)

    for index, row in slot.iterrows():
        warehouse_id = row['warehouse_id']
        slot_start_time = row['pickup_start_time']
        slot_end_time = row['pickup_end_time']
        
        # Find the end time of the previous slot (if available)
        if index > 0:
            prev_slot_start_time = results_df.loc[index - 1, 'pickup_start_time']
        else:
            prev_slot_start_time = None
        
        # Filter df for current warehouse_id and pickups between previous slot's end time and current slot's start time
        if prev_slot_start_time:
            filtered_df = df[(df['warehouse_id'] == warehouse_id) & 
                             (df['created_date'] >= prev_slot_start_time) & 
                             (df['created_date'] < slot_start_time)]
        else:
            filtered_df = df[(df['warehouse_id'] == warehouse_id) & 
                             (df['created_date'] < slot_start_time)]

        # Count the number of awbs within the slot time range
        expected_load = len(filtered_df)
        
        # Append the expected load count to the list
        expected_loads.append(expected_load)

# Add the expected_loads list as a new column to results_df
results_df['expected_load'] = expected_loads

# Display the updated results_df with expected load column
print(results_df)

# Save DataFrame to Excel
excel_file = 'expected_load.xlsx'
results_df.to_excel(excel_file, index=False)

print(f"DataFrame saved to {excel_file}")

     warehouse_id   pickup_start_time     pickup_end_time  slot_duration  \
0              83 2024-07-06 16:49:33 2024-07-06 16:55:17              6   
1             314 2024-07-06 01:40:19 2024-07-06 02:06:12             26   
2             314 2024-07-06 15:42:51 2024-07-06 16:06:08             24   
3             314 2024-07-06 17:37:28 2024-07-06 17:45:11              8   
4             327 2024-07-06 18:49:49 2024-07-06 18:50:15              1   
..            ...                 ...                 ...            ...   
214          2142 2024-07-06 17:53:20 2024-07-06 17:53:58              1   
215          2143 2024-07-06 18:54:56 2024-07-06 19:07:06             13   
216          2144 2024-07-06 18:54:05 2024-07-06 19:07:28             14   
217          2147 2024-07-06 12:23:09 2024-07-06 12:25:42              3   
218          2151 2024-07-06 16:46:59 2024-07-06 16:47:04              1   

     gap_to_next  load  actual_load  expected_load  
0            NaN    47           4